
# Algorithmic Trading Machine Learning Project

This is a machine learning project that will take in S&P 500 stock price data and use an unsupervised learning trading strategy. This is strictly for educational purposes and is to help me learn machine learning.

1. Install packages.

In [33]:
# Packages installed:
# pandas
# pandas_ta
# numpy
# matplotlib
# statsmodels
# pandas_datareader
# datetime
# yfinance
# sklearn
# PyPortfolioOpt

!pip install pandas pandas_ta numpy matplotlib statsmodels pandas_datareader datetime yfinance sklearn PyPortfolioOpt -q


You should consider upgrading via the 'c:\users\kyle\onedrive\desktop\project folders\algoirthmic trading\my_virtual_env\scripts\python.exe -m pip install --upgrade pip' command.


2. Download S&P 500 data

In [51]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

# Read the S&P 500 list of companies
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

# Remove the dot from the symbol names
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

# Get the list of symbols
symbols_list = sp500['Symbol'].unique().tolist()
symbols_list

# Define the start and end dates
end_date = '2023-09-27'
start_date = pd.to_datetime(end_date) - pd.DateOffset(365 * 8)

# Download the data
df = yf.download(tickers = symbols_list, 
                 start=start_date, 
                 end=end_date).stack()

df.index.names = ['date', 'ticker']
df.columns = df.columns.str.lower()


[*********************100%%**********************]  503 of 503 completed

3 Failed downloads:
['SOLV', 'GEV', 'VLTO']: YFChartError("%ticker%: Data doesn't exist for startDate = 1443499200, endDate = 1695787200")


2. Technical indicators for all the stocks

In [110]:
# Garman-Klass volatility
df['garman_klass_vol'] = ((np.log(df['high']) - np.log(df['low'])) **2/2 - ((2 * np.log(2) - 1) * (np.log(df['adj close']) - np.log(df['open'])) ** 2))

# RSI
df['rsi'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.rsi(close = x, length = 20))

# Bolinger Bands
df['bb_low'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.bbands(close = np.log1p(x), length = 20).iloc[:, 0])
df['bb_med'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.bbands(close = np.log1p(x), length = 20).iloc[:, 1])
df['bb_high'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.bbands(close = np.log1p(x), length = 20).iloc[:, 2])

# ATR
def compute_atr(stock_data):
    atr = pandas_ta.atr(high = stock_data['high'],
                        low = stock_data['low'],
                        close = stock_data['adj close'],
                        length = 14)
    return atr.sub(atr.mean()).div(atr.std())
df['atr'] = df.groupby(level = 1, group_keys = False).apply(compute_atr)

# MACD
def compute_macd(close):
    macd = pandas_ta.macd(close = close, length = 20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level = 1, group_keys = False)['adj close'].apply(compute_macd)

# Volume
df['dollar_vol'] = (df['adj close'] * df['volume']) / 1e6
df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2015-09-29 A        31.425230   33.740002   34.060001   33.240002   33.360001   
           AAL      37.361618   39.180000   39.770000   38.790001   39.049999   
           AAPL     24.651136   27.264999   28.377501   26.965000   28.207500   
           ABBV     36.334896   52.790001   54.189999   51.880001   53.099998   
           ABT      33.478710   39.500000   40.150002   39.029999   39.259998   
...                       ...         ...         ...         ...         ...   
2023-09-26 XYL      88.736305   89.519997   90.849998   89.500000   90.379997   
           YUM     122.211006  124.010002  124.739998  123.449997  124.239998   
           ZBH     111.534821  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZTS     175.547714  176.869995  178.449997  176.270004  176.580002   

Price                   volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2015-09-29 A         2252400.0         -0.001082        NaN       NaN   
           AAL       7478800.0         -0.000443        NaN       NaN   
           AAPL    293461600.0         -0.005712        NaN       NaN   
           ABBV     12842800.0         -0.054656        NaN       NaN   
           ABT      12287500.0         -0.009402        NaN       NaN   
...                        ...               ...        ...       ...   
2023-09-26 XYL       1322400.0         -0.000018  26.146735  4.485761   
           YUM       1500600.0         -0.000051  36.057175  4.811707   
           ZBH       3610500.0          0.000022  31.893224  4.745884   
           ZBRA       355400.0          0.000133  29.494977  5.400991   
           ZTS       1463200.0          0.000062  42.623488  5.156108   

Price                bb_med   bb_high       atr      macd   dollar_vol  
date       ticker                                                       
2015-09-29 A            NaN       NaN       NaN       NaN    70.782188  
           AAL          NaN       NaN       NaN       NaN   279.420069  
           AAPL         NaN       NaN       NaN       NaN  7234.161929  
           ABBV         NaN       NaN       NaN       NaN   466.641803  
           ABT          NaN       NaN       NaN       NaN   411.369651  
...                     ...       ...       ...       ...          ...  
2023-09-26 XYL     4.567684  4.649607 -2.967042 -2.159188   117.344890  
           YUM     4.841672  4.871637 -2.813230 -1.363696   183.389836  
           ZBH     4.785551  4.825217 -2.109952 -0.881067   402.696470  
           ZBRA    5.539167  5.677342 -0.057389 -1.600791    79.595386  
           ZTS     5.214922  5.273735 -0.720920 -1.188278   256.861415  

[987967 rows x 14 columns]

3. Filter top 150 most liquid

In [123]:
# Define the list of columns to keep
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_vol', 'volume', 'open', 'high', 'low', 'close']]

data = pd.concat([df.unstack('ticker')['dollar_vol'].resample('M').mean().stack('ticker').to_frame('dollar_vol'),
df.unstack()[last_cols].resample('M').last().stack('ticker')], axis = 1).dropna()

data

dollar_vol   adj close       atr   bb_high    bb_low  \
date       ticker                                                          
2015-11-30 A        135.740909   38.950897 -0.948797  3.689080  3.544191   
           AAL      287.915785   39.429939  1.473839  3.827635  3.672028   
           AAPL    4023.984070   26.854139 -0.326059  3.368302  3.281679   
           ABBV     337.563903   40.393524  0.572009  3.823191  3.726695   
           ABT      211.659048   38.293568  0.553865  3.699763  3.656056   
...                        ...         ...       ...       ...       ...   
2023-09-30 OTIS     154.361752   78.356506 -2.269283  4.460712  4.370137   
           ABNB    1633.500725  132.279999 -1.006939  5.024801  4.857047   
           CEG      196.670369  107.862030 -0.705546  4.732493  4.652147   
           GEHC     212.275850   66.130219 -0.905989  4.270508  4.155436   
           KVUE     670.804320   20.006195 -2.650790  3.164168  3.014275   

                     bb_med  garman_klass_vol      macd        rsi  
date       ticker                                                   
2015-11-30 A       3.616636         -0.002098  0.567157  73.421554  
           AAL     3.749832         -0.000966 -0.418772  40.719000  
           AAPL    3.324991         -0.003307 -0.142789  55.537348  
           ABBV    3.774943         -0.059549  0.145677  49.376849  
           ABT     3.677909         -0.011216  0.335557  56.962557  
...                     ...               ...       ...        ...  
2023-09-30 OTIS    4.415425         -0.000097 -1.534536  33.116256  
           ABNB    4.940924          0.000213 -0.037854  44.494127  
           CEG     4.692320          0.000131  0.366876  55.245482  
           GEHC    4.212972          0.000185 -1.116463  40.922337  
           KVUE    3.089221         -0.000159 -1.435621  35.706312  

[46648 rows x 9 columns]

In [124]:
# Rolling average 5 year
data['dollar_vol'] = (data['dollar_vol'].unstack('ticker').rolling(5*12).mean().stack())

# Rank the stocks by dollar volume
data['dollar_vol_rank'] = (data.groupby('date')['dollar_vol'].rank(ascending=False))

# Keep only the top 150 stocks by dollar volume
data = data[data['dollar_vol_rank']<150].drop(['dollar_vol', 'dollar_vol_rank'], axis=1)

data

adj close       atr   bb_high    bb_low    bb_med  \
date       ticker                                                       
2020-10-31 AAL      11.280000 -1.164577  2.700490  2.483765  2.592127   
           AAPL    106.506020  1.603320  4.803695  4.686059  4.744877   
           ABBV     73.725426 -0.699048  4.352573  4.265789  4.309181   
           ABT      98.835152  1.065572  4.655820  4.601153  4.628486   
           ACN     206.515457 -0.091047  5.420623  5.322555  5.371589   
...                       ...       ...       ...       ...       ...   
2023-09-30 VRTX    351.690002  0.029799  5.879295  5.838959  5.859127   
           VZ       31.258692 -2.233454  3.532000  3.467585  3.499792   
           WFC      39.781086 -1.480694  3.777782  3.697054  3.737418   
           WMT      53.597538 -2.579274  4.019915  3.984976  4.002446   
           XOM     113.372101 -1.345888  4.767283  4.687091  4.727187   

                   garman_klass_vol      macd        rsi  
date       ticker                                         
2020-10-31 AAL             0.000622 -0.348861  41.027057  
           AAPL            0.000078 -0.628203  43.770322  
           ABBV           -0.004957 -0.994068  47.517640  
           ABT            -0.001040 -0.069483  48.047134  
           ACN            -0.000481 -0.949112  42.280855  
...                             ...       ...        ...  
2023-09-30 VRTX            0.000037  0.027907  52.406728  
           VZ             -0.000940 -0.350386  42.222474  
           WFC            -0.000110 -0.282326  40.920275  
           WMT            -0.000041  0.399459  54.722524  
           XOM            -0.000065  1.400623  59.440190  

[5364 rows x 8 columns]

In [131]:
# Calculate the returns function
def calculate_returns(df):
    outlier_cutoff = 0.005
    lags = [1,2,3,6,9,12]

    for lag in lags:
        df[f'return_{lag}m'] = df['adj close'].pct_change(lag).pipe(lambda x: x.clip(lower = x.quantile(outlier_cutoff), upper = x.quantile(1-outlier_cutoff))).add(1).pow(1/lag).sub(1)
    return df
data = data.groupby(level = 1, group_keys = False).apply(calculate_returns).dropna()
data



adj close       atr   bb_high    bb_low    bb_med  \
date       ticker                                                       
2022-10-31 AAL      14.180000 -1.174089  2.758665  2.562969  2.660817   
           AAPL    151.835175  1.798464  5.040184  4.908286  4.974235   
           ABBV    138.093994 -1.213245  4.984791  4.853662  4.919227   
           ABT      96.056801 -1.311948  4.637995  4.533595  4.585795   
           ACN     277.369263 -1.119774  5.650465  5.487150  5.568807   
...                       ...       ...       ...       ...       ...   
2023-09-30 VRTX    351.690002  0.029799  5.879295  5.838959  5.859127   
           VZ       31.258692 -2.233454  3.532000  3.467585  3.499792   
           WFC      39.781086 -1.480694  3.777782  3.697054  3.737418   
           WMT      53.597538 -2.579274  4.019915  3.984976  4.002446   
           XOM     113.372101 -1.345888  4.767283  4.687091  4.727187   

                   garman_klass_vol      macd        rsi  return_1m  \
date       ticker                                                     
2022-10-31 AAL             0.000421  0.441920  56.108827   0.177741   
           AAPL            0.000086 -0.037803  53.743134   0.109551   
           ABBV           -0.000792  1.338599  53.847309   0.101927   
           ABT            -0.000290 -1.107027  46.686617   0.027355   
           ACN            -0.000050  0.652385  56.897277   0.108288   
...                             ...       ...        ...        ...   
2023-09-30 VRTX            0.000037  0.027907  52.406728   0.009617   
           VZ             -0.000940 -0.350386  42.222474  -0.056890   
           WFC            -0.000110 -0.282326  40.920275  -0.015500   
           WMT            -0.000041  0.399459  54.722524  -0.000676   
           XOM            -0.000065  1.400623  59.440190   0.046947   

                   return_2m  return_3m  return_6m  return_9m  return_12m  
date       ticker                                                          
2022-10-31 AAL      0.044801   0.011299  -0.045662  -0.016497   -0.024940  
           AAPL    -0.012417  -0.018720  -0.004135  -0.013984    0.002415  
           ABBV     0.048751   0.010071   0.002697   0.010618    0.023821  
           ABT     -0.015924  -0.029767  -0.021131  -0.026333   -0.020366  
           ACN     -0.005732  -0.023511  -0.008030  -0.022903   -0.018213  
...                      ...        ...        ...        ...         ...  
2023-09-30 VRTX    -0.000923  -0.000208   0.018495   0.022140    0.016337  
           VZ      -0.016122  -0.033458  -0.021495  -0.014100   -0.006327  
           WFC     -0.057917  -0.013554   0.016441   0.000702    0.003255  
           WMT      0.010014   0.012354   0.017574   0.016553    0.020256  
           XOM      0.046139   0.030496   0.012838   0.008747    0.027037  

[1656 rows x 14 columns]